In [1]:
import pandas as pd
import numpy as np
import gc
import os
import sys
import copy
import transforms3d
import matplotlib.pyplot as plt
import open3d as o3d
from tqdm import tqdm_notebook
from sklearn.neighbors import NearestNeighbors

In [2]:
BASE_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1"
PAIR_FILE = "pair_with_pose.csv"
pair_fname = os.path.join(BASE_PATH, PAIR_FILE)

df_pair = pd.read_csv(pair_fname)

In [3]:
df_pair.shape

(46140, 5)

In [4]:
df_pair.head()

,static_path,dynamic_path,pose_x,pose_y,pose_yaw
0,/home/sabyasachi/Projects/ati/data/data/datase...,/home/sabyasachi/Projects/ati/data/data/datase...,0.034119,-0.363983,0.003305
1,/home/sabyasachi/Projects/ati/data/data/datase...,/home/sabyasachi/Projects/ati/data/data/datase...,0.034119,-0.363983,0.003305
2,/home/sabyasachi/Projects/ati/data/data/datase...,/home/sabyasachi/Projects/ati/data/data/datase...,-0.010010,-0.363281,-0.002533
3,/home/sabyasachi/Projects/ati/data/data/datase...,/home/sabyasachi/Projects/ati/data/data/datase...,0.014374,-0.166115,-0.007080
4,/home/sabyasachi/Projects/ati/data/data/datase...,/home/sabyasachi/Projects/ati/data/data/datase...,0.337585,-0.282562,1.754612


In [5]:
def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

def score_pts(src_pcd, dst_pcd, trans_arr=np.zeros(3), rot_ang=np.zeros(3)):
    NEIGHB_RADIUS = 0.1
    
    src_tmp_pcd = copy.deepcopy(src_pcd)
    dst_tmp_pcd = copy.deepcopy(dst_pcd)
    
    transform_mat = pose2matrix(trans_arr, rot_ang)
    # Note: we apply transformation on target pcd to directly get pose (without sign change)
    dst_tmp_pcd.transform(transform_mat)
    
    score = o3d.registration.evaluate_registration(src_tmp_pcd, dst_tmp_pcd, NEIGHB_RADIUS)
    
    fit = score.fitness
    rmse = score.inlier_rmse
    n_pairs = np.asarray(score.correspondence_set).shape[0]
    return rmse, fit, n_pairs

def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [ ]:
for idx, row in df_pair.iterrows():
    src_pcd = o3d.io.read_point_cloud(row['static_path'])
    dst_pcd = o3d.io.read_point_cloud(row['dynamic_path'])
    score_pts(src_pcd, dst_pcd, trans_arr=np.array([row['pose_x'], row['pose_y'],0]), rot_ang=np.array([0,0,row['pose_yaw']]))

In [45]:
some_row = df_pair.iloc[np.random.choice(df_pair.shape[0])]
# some_row = df_pair.iloc[2]

print("Distance (in m): {} \nRelative Angle (in deg): {}".format(np.linalg.norm(np.array([some_row['pose_x'],some_row['pose_y']])-np.array([0,0])), some_row['pose_yaw']))

some_static_pcd = o3d.io.read_point_cloud(some_row['static_path'])
some_dynamic_pcd = o3d.io.read_point_cloud(some_row['dynamic_path'])

draw_registration_result(some_static_pcd, some_dynamic_pcd, 0, 0, 0)
# Static is red
# dynamic is blue

Distance (in m): 0.21236680284225087 
Relative Angle (in deg): -0.001708984375


JVisualizer with 2 geometries

In [47]:
draw_registration_result(some_static_pcd, some_dynamic_pcd, -some_row['pose_x'], some_row['pose_y'], some_row['pose_yaw'])

JVisualizer with 2 geometries